In [2]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import OrderedDict

# Most Influential Artists (Highest Out-Degree in AllMusic Graph)

In [39]:
# Load artist information
artists = pd.read_csv('data/allmusic/artists.txt', header=None, names=['name', 'url', 'active_period', 'genres', 'styles'])
# Load influence relationships
influences = pd.read_csv('data/allmusic/influences.txt', header=None, names=['influencer_name', 'influencer_url', 'follower_name', 'follower_url'])

In [64]:
# Create directed graph from influence relationships
G = nx.DiGraph()

for row in influences.itertuples():
    G.add_edge(row.influencer_name, row.follower_name)

In [83]:
# Find nodes with highest outdegree
outdegree_dict = G.out_degree()
# Display 100 most influential artists
sorted(outdegree_dict.items(), key=lambda x: x[1], reverse=True)[:100]

[('The Beatles', 829),
 ('Bob Dylan', 540),
 ('The Rolling Stones', 427),
 ('David Bowie', 317),
 ('The Velvet Underground', 314),
 ('The Kinks', 294),
 ('The Beach Boys', 282),
 ('Jimi Hendrix', 281),
 ('Neil Young', 254),
 ('James Brown', 251),
 ('The Byrds', 247),
 ('Led Zeppelin', 246),
 ('Hank Williams', 238),
 ('Miles Davis', 234),
 ('John Coltrane', 222),
 ('Brian Eno', 215),
 ('Black Sabbath', 213),
 ('Elvis Presley', 212),
 ('Chuck Berry', 204),
 ('The Who', 203),
 ('Pink Floyd', 197),
 ('Kraftwerk', 189),
 ('The Clash', 189),
 ('The Stooges', 189),
 ('Sex Pistols', 187),
 ('Marvin Gaye', 186),
 ('Sonic Youth', 186),
 ('Stevie Wonder', 180),
 ('Muddy Waters', 171),
 ('Ramones', 167),
 ('Johnny Cash', 163),
 ('Duke Ellington', 161),
 ('Louis Armstrong', 149),
 ('Nirvana', 146),
 ('Sam Cooke', 146),
 ('Charlie Parker', 146),
 ('Joni Mitchell', 146),
 ('Prince', 140),
 ('Black Flag', 135),
 ('Metallica', 135),
 ('Leonard Cohen', 134),
 ('R.E.M.', 132),
 ('Roxy Music', 132),
 ('Bi

In [88]:
# Write Gephi file
nx.write_edgelist(G, 'all_influence')